In [51]:
from stellar_sdk import Account, Asset, Keypair, Network, TransactionBuilder, Server
import networkx as nx
        from datetime import datetime
        from pprint import pprint
        import math
        from decimal import Decimal
        from copy import deepcopy
import bisect 
server = Server(horizon_url="https://horizon-testnet.stellar.org")

In [52]:
def create_random_keypairs(num=30):
    return [Keypair.random() for _ in range(num)]

def create_accounts(keypairs, secret):
    signer = Keypair.from_secret(secret)
    account = server.load_account(signer.public_key)
    
    max_ops_per_tx = 100
    num_batches = math.ceil(len(keypairs) / max_ops_per_tx)
        
    for batch_i in range(num_batches):
        start = batch_i * max_ops_per_tx
        end = batch_i * max_ops_per_tx + max_ops_per_tx
        
        print(f"create_accounts - {batch_i + 1}/{num_batches}, start: {start}, end: {end}")
        keypairs_batch = keypairs[start:end]
        
        tx = TransactionBuilder(
            source_account=account,
            network_passphrase=Network.TESTNET_NETWORK_PASSPHRASE,
            base_fee=100,
        )
        
        for keypair in keypairs_batch:
            tx.append_create_account_op(
                destination=keypair.public_key,
                starting_balance="10"
            )
        tx = tx.set_timeout(30).build()    
        tx.sign(signer)
        response = server.submit_transaction(tx)
        print(f'successful: {response["successful"]}')
    

def create_trustlines(keypairs, custom_asset):
    max_signers_per_tx = 20
    num_batches = math.ceil(len(keypairs) / max_signers_per_tx)
    
    for batch_i in range(num_batches):
        start = batch_i * max_signers_per_tx
        end = batch_i * max_signers_per_tx + max_signers_per_tx
        
        print(f"create_trustlines - {batch_i + 1}/{num_batches}, start: {start}, end: {end}")
        keypairs_batch = keypairs[start:end]
        
        signer = keypairs_batch[0]
        account = server.load_account(signer.public_key)
        
        tx = TransactionBuilder(
            source_account=account,
            network_passphrase=Network.TESTNET_NETWORK_PASSPHRASE,
            base_fee=100,
        )
        
        for keypair in keypairs_batch:
            tx.append_change_trust_op(
                source=keypair.public_key,
                asset=custom_asset,
                limit=None
            )
    
        tx = tx.set_timeout(30).build()

        for kp in keypairs_batch:
            tx.sign(kp)
        response = server.submit_transaction(tx)
        print(f'successful: {response["successful"]}')
        
def fund_accounts(keypairs, secret, custom_asset):
    signer = Keypair.from_secret(secret)
    account = server.load_account(signer.public_key)
    
    max_ops_per_tx = 100
    num_batches = math.ceil(len(keypairs) / max_ops_per_tx)
    
    for batch_i in range(num_batches):
        start = batch_i * max_ops_per_tx
        end = batch_i * max_ops_per_tx + max_ops_per_tx
        
        print(f"fund_accounts - {batch_i + 1}/{num_batches}, start: {start}, end: {end}")
        keypairs_batch = keypairs[start:end]
        
        tx = TransactionBuilder(
            source_account=account,
            network_passphrase=Network.TESTNET_NETWORK_PASSPHRASE,
            base_fee=100,
        )
        
        for keypair in keypairs_batch:
            tx.append_payment_op(
                destination=keypair.public_key,
                asset=custom_asset,
                amount="100"
            )
        tx = tx.set_timeout(30).build()
        
        tx.sign(signer)

        response = server.submit_transaction(tx)
        print(f'successful: {response["successful"]}')

        
def create_payments(keypairs, asset):  
    dims = [3, 1]
    
    signer = keypairs[0]
    account = server.load_account(signer.public_key)

    tx = TransactionBuilder(
        source_account=account,
        network_passphrase=Network.TESTNET_NETWORK_PASSPHRASE,
        base_fee=100,
    )
    
    signers = []
    
    for i in range(dims[0]):
        for j in range(dims[1]):
            source_i = (i * dims[1]) + j
            dest_i = (((i + 1) * dims[1]) + j) % (dims[0] * dims[1])
            amount = "0.001" if i == 0 and j == 0 else f"0.{i}{j}"
            
            print(f"source_i: {source_i},  dest_i: {dest_i}, amount: {amount}")
            
            source_pk = keypairs[source_i].public_key
            dest_pk = keypairs[dest_i].public_key
            
            print(f"{source_pk[-4:]} -> {dest_pk[-4:]}")
            
            tx.append_payment_op(
                source=source_pk,
                destination=dest_pk,
                asset=asset,
                amount=amount
            )
            
            if source_i != 0:
                signers.append(keypairs[source_i])
            
    tx = tx.set_timeout(30).build()
    for _signer in signers:
        tx.sign(_signer) 
    tx.sign(signer)
    response = server.submit_transaction(tx)
    print(f'successful: {response["successful"]}')
    
def pay(source_secret, destination_public, amount, asset):
    source = Keypair.from_secret(source_secret)
    account = server.load_account(source.public_key)
    
    tx = TransactionBuilder(
        source_account=account,
        network_passphrase=Network.TESTNET_NETWORK_PASSPHRASE,
        base_fee=100,
    ).append_payment_op(
        destination=destination_public,
        asset=asset,
        amount=amount
    ).set_timeout(30).build()
    
    tx.sign(source)
    response = server.submit_transaction(tx)
    print(f'successful: {response["successful"]}, created_at: {response["created_at"]}')

def timestring_to_timestamp(timestring):
    return datetime.strptime(timestring, "%Y-%m-%dT%H:%M:%SZ").timestamp() 
            

In [35]:
issuer_secret = "SDPFNKPPYEMA6QPY3ZBO3NA2SUR7I4OHBOCOCF2UDQSLGRWYRF7CO7GJ"

my_asset = Asset("A6", "GCFRQ3CQG5XY4CH5E4ZSTACZ27YYFEWZG636J4GIBWY3FKE3KSJ5ZBCK")

keypairs = create_random_keypairs(3)
create_accounts(keypairs, issuer_secret)
create_trustlines(keypairs, my_asset)
fund_accounts(keypairs, issuer_secret, my_asset)

pay(keypairs[0].secret, keypairs[1].public_key, "10", my_asset)
pay(keypairs[1].secret, keypairs[2].public_key, "20", my_asset)
pay(keypairs[2].secret, keypairs[0].public_key, "30", my_asset)

snapshot = create_balance_snapshot(
    my_asset.code, 
    my_asset.issuer, 
    timestamp=None
)

snapshot_0_test = {
    keypairs[0].public_key: 120,
    keypairs[1].public_key: 90,
    keypairs[2].public_key: 90,
}

assert(snapshot_0_test[keypairs[0].public_key] == snapshot[keypairs[0].public_key])
assert(snapshot_0_test[keypairs[1].public_key] == snapshot[keypairs[1].public_key])
assert(snapshot_0_test[keypairs[2].public_key] == snapshot[keypairs[2].public_key])


pay(keypairs[1].secret, keypairs[2].public_key, "20", my_asset)
pay(keypairs[2].secret, keypairs[0].public_key, "20", my_asset)
pay(keypairs[0].secret, keypairs[2].public_key, "20", my_asset)

create_accounts - 1/1, start: 0, end: 100
successful: True
create_trustlines - 1/1, start: 0, end: 20
successful: True
fund_accounts - 1/1, start: 0, end: 100
successful: True
successful: True, created_at: 2023-01-24T13:48:14Z
successful: True, created_at: 2023-01-24T13:48:20Z
successful: True, created_at: 2023-01-24T13:48:26Z
ops_to_check
[('2908693586776065', 1674564488.0),
 ('2908693586776066', 1674564488.0),
 ('2908693586776067', 1674564488.0)]
checked_ops
[]
OPS_TO_CHECK
[('2908693586776067', 1674564488.0), ('2908693586776066', 1674564488.0), ('2908693586776065', 1674564488.0)]
1674564488.0
2908693586776065 GCFRQ3CQG5XY4CH5E4ZSTACZ27YYFEWZG636J4GIBWY3FKE3KSJ5ZBCK GCGBLLQLSAGOA4BWU6Z3NQHWEOGQHVUH7N5OOO5QBTTTQEXERY4YNRAA 100.0000000 0 0
{'GCFRQ3CQG5XY4CH5E4ZSTACZ27YYFEWZG636J4GIBWY3FKE3KSJ5ZBCK': Decimal('-100.0000000'),
 'GCGBLLQLSAGOA4BWU6Z3NQHWEOGQHVUH7N5OOO5QBTTTQEXERY4YNRAA': Decimal('100.0000000')}
Operation 2908689291812865 - unknown type: change_trust
ops_to_check: 4
[('2908

In [26]:
pay(keypairs[0].secret, keypairs[1].public_key, "20", my_asset)

successful: True, created_at: 2023-01-24T13:18:04Z


In [15]:
pay(keypairs[1].secret, keypairs[2].public_key, "6", my_asset)

successful: True


In [18]:
pay(keypairs[2].secret, keypairs[0].public_key, "7", my_asset)

successful: True


In [25]:
for k in keypairs:
    print(k.public_key)

GARQQ6HLN4CCT5JS6552XKYECKVDETV4JWVP542EGYGENPB6TG4BFHMQ
GAZLVYQIQUTVJMRXXV2IKXC6FWUXTE6QT4I4PKSZ3TCCLO5VBAZXTVML
GCGPWKTD2IKSLU4V2G46UFEX4HPLNE4ZLLISXTYPB3J3GOSRTVVYHANC


In [73]:
snapshots = create_balance_snapshots(
    my_asset.code, 
    my_asset.issuer, 
    timestamp=None #timestring_to_timestamp("2023-01-24T13:17:26Z")
)

for key, snapshot in snapshots.items():
    print("~~~~~~~~~~~~~~")
    print("TIME:\n", key)
    print("\nBALANCES:")
    pprint(snapshot["balances"])
    print("\nTRANSACTIONS:")
    pprint(snapshot["transactions"])
    #pprint(snapshot)

~~~~~~~~~~~~~~
TIME:
 2023-01-24T13:48:08Z

BALANCES:
{'GB5OB5WF3547Y444VCVEOB4DUY4O637NI7XCN46FSEINCGF657ZF6VXK': Decimal('100.0000000'),
 'GBNBF7333WTP3AXZTNRKL563UWQU6EFPPWNUWH7YDLQKH76FQIRI7KKS': Decimal('100.0000000'),
 'GCFRQ3CQG5XY4CH5E4ZSTACZ27YYFEWZG636J4GIBWY3FKE3KSJ5ZBCK': Decimal('-300.0000000'),
 'GCGBLLQLSAGOA4BWU6Z3NQHWEOGQHVUH7N5OOO5QBTTTQEXERY4YNRAA': Decimal('100.0000000')}

TRANSACTIONS:
[('GCFRQ3CQG5XY4CH5E4ZSTACZ27YYFEWZG636J4GIBWY3FKE3KSJ5ZBCK',
  'GCGBLLQLSAGOA4BWU6Z3NQHWEOGQHVUH7N5OOO5QBTTTQEXERY4YNRAA',
  '100.0000000'),
 ('GCFRQ3CQG5XY4CH5E4ZSTACZ27YYFEWZG636J4GIBWY3FKE3KSJ5ZBCK',
  'GBNBF7333WTP3AXZTNRKL563UWQU6EFPPWNUWH7YDLQKH76FQIRI7KKS',
  '100.0000000'),
 ('GCFRQ3CQG5XY4CH5E4ZSTACZ27YYFEWZG636J4GIBWY3FKE3KSJ5ZBCK',
  'GB5OB5WF3547Y444VCVEOB4DUY4O637NI7XCN46FSEINCGF657ZF6VXK',
  '100.0000000')]
~~~~~~~~~~~~~~
TIME:
 2023-01-24T13:48:14Z

BALANCES:
{'GB5OB5WF3547Y444VCVEOB4DUY4O637NI7XCN46FSEINCGF657ZF6VXK': Decimal('110.0000000'),
 'GBNBF7333WTP3AXZTNRKL56

In [63]:
def create_balance_snapshots(asset_code, asset_issuer, timestamp):

    snapshots = {}
    balances = {}
    ops_to_check = []
    checked_ops = []
    
    ops_to_check = get_initial_ops_to_check(asset_code, asset_issuer)
                
    while len(ops_to_check) > 0:
        #optimalization nlog(n) to n: keeping a sorted list by sorted insert
        ops_to_check.sort(key=lambda x: x[1])
        ops_to_check.reverse()
        op_id, op_timestamp = ops_to_check.pop()
        op = server.operations().operation(op_id).order(desc=True).call()
        
        created_at_ts = timestring_to_timestamp(op["created_at"])
        if timestamp is not None and created_at_ts > timestamp:
            checked_ops.append(op["id"])
            continue

        if op["type"] == "payment":

            if op["id"] not in checked_ops:
                from_pk = op["from"]
                to_pk = op["to"]
                amount = op["amount"]
                
                op_timestamp = timestring_to_timestamp(op["created_at"])

                if from_pk not in balances:
                    balances[from_pk] = Decimal("0")

                if to_pk not in balances:
                    balances[to_pk] = Decimal("0")
                    
                #print(op["id"], from_pk, to_pk, amount, balances[from_pk], balances[to_pk])
                

                balances[from_pk] -= Decimal(amount)
                balances[to_pk] += Decimal(amount)
                
                if op["created_at"] not in snapshots:
                    snapshots[op["created_at"]] = {"balances": {}, "transactions": []}
                    
                snapshots[op["created_at"]]["balances"] = deepcopy(balances)
                snapshots[op["created_at"]]["transactions"].append((from_pk, to_pk, amount))
                
                #pprint(balances)
                checked_ops.append(op["id"]) 
                
                call_builder = server.operations().for_account(to_pk).order(desc=False)
                records = ops_page = call_builder.call()["_embedded"]["records"]
                while records_page := call_builder.next()["_embedded"]["records"]: 
                    records += records_page
                
                for record in records:
                    if ("asset_code" in record and "asset_issuer" in record and
                        record["asset_code"] == asset_code and record["asset_issuer"] == asset_issuer):
                        if record["type"] == "payment":
                            
                            ids_to_check = [x[0] for x in ops_to_check]
                            if record["id"] not in checked_ops and record["id"] not in ids_to_check:
                                op_to_check = (
                                    record["id"],
                                    timestring_to_timestamp(record["created_at"])
                                )
                                ops_to_check.append(op_to_check)
                        else:
                            pass
                            

    return snapshots

def get_initial_ops_to_check(asset_code, asset_issuer):

    ops_to_check = []
    
    call_builder = server.operations().for_account(asset_issuer).order(desc=False)

    records = call_builder.call()["_embedded"]["records"]

    while page_records := call_builder.next()["_embedded"]["records"]: 
        records += page_records
    
    for record in records:
        if ("asset_code" in record and record["asset_code"] == asset_code and
            "asset_issuer" in record and record["asset_issuer"] == asset_issuer):

            if record["type"] == "payment":
                op_to_check = (
                    record["id"],
                    timestring_to_timestamp(record["created_at"])
                )
                ops_to_check.append(op_to_check)

            else:
                print(f"Unknown type: {record['type']}")
                    
    return ops_to_check
    


In [2]:
accounts = [
    {
        "public": "GDOSYLFJONVW65YU4YB2ZGR2FOLPT23B5MCXXGTVSMBQQPJCALJK4VW4",
        "secret": "SDOW25QP7U3X7HKTQYPUJTBRBGVNSU37L3HYF5SGPNJUUDMDOI7JZJ74"
    },
    {
         "public": "GCFRQ3CQG5XY4CH5E4ZSTACZ27YYFEWZG636J4GIBWY3FKE3KSJ5ZBCK",
         "secret": "SDPFNKPPYEMA6QPY3ZBO3NA2SUR7I4OHBOCOCF2UDQSLGRWYRF7CO7GJ"
    },
    {
        "public": "GCOAWTQ2F3IXVEOAGGN2Q7UZTZWI7SBSHF6QA3W5HAO7CFHS7WBAYSWZ",
        "secret": "SDRZT6W7LLUCDHDBSSW32S3FP22EEW2D52IUZAICRFODT2SLMTXC52LS"
    },
    {
        "public": "GD6I4POIIXE6S5AZMJG6GXIAH4KE3D5YBEBCACD2TT4XRZEUFITLCRFJ",
        "secret": "SDVWHCGYJMKB7U7YNXIHKGK2CZLEEXEW6D45Y7F2IYZXY2IEQUXGHNYZ"
    },
    {
        "public": "GBCJ4B4WK2NWUYPSOROVQOKBODYSYADT6SKLQBOCXBU4KQCV4M4ER36J",
        "secret": "SBOU7GLW47YLR2CBC7HI6Q6JY2E6TYNRT6CM7MB2J5QTMMYOETEFQOJL"
    }
]

In [29]:

def construct_graph(pk=accounts[0]["public"], asset_code="TestAsset6", visited_ops=[], G=nx.DiGraph()):
    
    call_builder = server.effects().for_account(pk).order(desc=False)

    for record in call_builder.call()["_embedded"]["records"]:
        op_id = record["_links"]["operation"]["href"].split("/")[-1]

        print(record["type"], record["amount"], record["asset_code"], op_id)

        ops_call_builder = server.operations().operation(op_id).order(desc=False)
        op = ops_call_builder.call()
        
        
        #print(record)
        print(op["id"], visited_ops)
        if op["id"] in visited_ops:
            print("return")
            return

        elif op["type"] == "payment":
            #print(op["from"],"\n", op["to"], "\n", record["amount"])
            
            G.add_edge(
                op["from"], 
                op["to"], 
                weight=record["amount"], 
                timestamp=record["created_at"], 
                op_id=op["id"]
            )
            
            visited_ops.append(op["id"])
            construct_graph(op["to"], asset_code, visited_ops, G)
            
        else:
            print("UNHANDLED TYPE:", op["type"])
    
    return G

In [30]:
G = construct_graph()

KeyError: 'amount'

In [31]:
#nx.draw_networkx(G)
nx.draw(G, pos=nx.spring_layout(G), with_labels=True) 

NameError: name 'G' is not defined

In [32]:
G = nx.Graph()
G.add_edge("GDOSYLFJONVW65YU4YB2ZGR2FOLPT23B5MCXXGTVSMBQQPJCALJK4VW4","GCFRQ3CQG5XY4CH5E4ZSTACZ27YYFEWZG636J4GIBWY3FKE3KSJ5ZBCK")

In [33]:


G.get_edge_data("GDOSYLFJONVW65YU4YB2ZGR2FOLPT23B5MCXXGTVSMBQQPJCALJK4VW4", "GCFRQ3CQG5XY4CH5E4ZSTACZ27YYFEWZG636J4GIBWY3FKE3KSJ5ZBCK")

{}

In [95]:
create_balance_snapshot("TestAsset7", accounts[0]["public"], timestamp=None)

2861350162272257 GBCJ4B4WK2NWUYPSOROVQOKBODYSYADT6SKLQBOCXBU4KQCV4M4ER36J GDOSYLFJONVW65YU4YB2ZGR2FOLPT23B5MCXXGTVSMBQQPJCALJK4VW4 10.0000000 0 0
{'GBCJ4B4WK2NWUYPSOROVQOKBODYSYADT6SKLQBOCXBU4KQCV4M4ER36J': -10.0,
 'GDOSYLFJONVW65YU4YB2ZGR2FOLPT23B5MCXXGTVSMBQQPJCALJK4VW4': 10.0}
2861332982411265 GDOSYLFJONVW65YU4YB2ZGR2FOLPT23B5MCXXGTVSMBQQPJCALJK4VW4 GCFRQ3CQG5XY4CH5E4ZSTACZ27YYFEWZG636J4GIBWY3FKE3KSJ5ZBCK 10.0000000 10.0 0
{'GBCJ4B4WK2NWUYPSOROVQOKBODYSYADT6SKLQBOCXBU4KQCV4M4ER36J': -10.0,
 'GCFRQ3CQG5XY4CH5E4ZSTACZ27YYFEWZG636J4GIBWY3FKE3KSJ5ZBCK': 10.0,
 'GDOSYLFJONVW65YU4YB2ZGR2FOLPT23B5MCXXGTVSMBQQPJCALJK4VW4': 0.0}
Operation 2861285737762817 - unknown type: change_trust
2861337277386753 GCFRQ3CQG5XY4CH5E4ZSTACZ27YYFEWZG636J4GIBWY3FKE3KSJ5ZBCK GCOAWTQ2F3IXVEOAGGN2Q7UZTZWI7SBSHF6QA3W5HAO7CFHS7WBAYSWZ 10.0000000 10.0 0
{'GBCJ4B4WK2NWUYPSOROVQOKBODYSYADT6SKLQBOCXBU4KQCV4M4ER36J': -10.0,
 'GCFRQ3CQG5XY4CH5E4ZSTACZ27YYFEWZG636J4GIBWY3FKE3KSJ5ZBCK': 0.0,
 'GCOAWTQ2F3IXVEOAGGN2Q7UZTZWI

{'GBCJ4B4WK2NWUYPSOROVQOKBODYSYADT6SKLQBOCXBU4KQCV4M4ER36J': 100.0,
 'GDOSYLFJONVW65YU4YB2ZGR2FOLPT23B5MCXXGTVSMBQQPJCALJK4VW4': -400.0,
 'GCFRQ3CQG5XY4CH5E4ZSTACZ27YYFEWZG636J4GIBWY3FKE3KSJ5ZBCK': 100.0,
 'GCOAWTQ2F3IXVEOAGGN2Q7UZTZWI7SBSHF6QA3W5HAO7CFHS7WBAYSWZ': 100.0,
 'GD6I4POIIXE6S5AZMJG6GXIAH4KE3D5YBEBCACD2TT4XRZEUFITLCRFJ': 100.0}

In [82]:
server.operations().operation("2844926207340545").call()

{'_links': {'self': {'href': 'https://horizon-testnet.stellar.org/operations/2844926207340545'},
  'transaction': {'href': 'https://horizon-testnet.stellar.org/transactions/636a04d5df27950f150430522b11c90c8f5773e35b372d494538c0789d13d63d'},
  'effects': {'href': 'https://horizon-testnet.stellar.org/operations/2844926207340545/effects'},
  'succeeds': {'href': 'https://horizon-testnet.stellar.org/effects?order=desc&cursor=2844926207340545'},
  'precedes': {'href': 'https://horizon-testnet.stellar.org/effects?order=asc&cursor=2844926207340545'}},
 'id': '2844926207340545',
 'paging_token': '2844926207340545',
 'transaction_successful': True,
 'source_account': 'GDOSYLFJONVW65YU4YB2ZGR2FOLPT23B5MCXXGTVSMBQQPJCALJK4VW4',
 'type': 'payment',
 'type_i': 1,
 'created_at': '2023-01-23T16:09:33Z',
 'transaction_hash': '636a04d5df27950f150430522b11c90c8f5773e35b372d494538c0789d13d63d',
 'asset_type': 'credit_alphanum12',
 'asset_code': 'TestAsset2',
 'asset_issuer': 'GDOSYLFJONVW65YU4YB2ZGR2FOLP

In [115]:
a = [0,1,2]

In [118]:
a.pop(), a

(0, [])

In [1]:
import bisect
x = [("a", 0), ("b", 1), ("d", 4), ("g", 6)]
y = ("c", 2)
bisect.insort(x, y, key=lambda a: a[1])


TypeError: 'key' is an invalid keyword argument for insort_right()